In [3]:
# import libraries
import pandas as pd
import glob

In [51]:
# use list comprehention and glob to create one db from many
# works because he each db has the same columns
df = pd.concat([pd.read_csv(x) for x in glob.glob('data/Kickstarter*.csv')], ignore_index = True)

In [5]:
df.head()

,backers_count,blurb,category,converted_pledged_amount,country,country_displayable_name,created_at,creator,currency,currency_symbol,...,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
0,61,Support great art! Join us as we re-stage the ...,"{""id"":254,""name"":""Performances"",""slug"":""dance/...",4618,US,the United States,1579292017,"{""id"":2092817311,""name"":""Brooklyn Ballet"",""slu...",USD,$,...,revisionist-history-2,https://www.kickstarter.com/discover/categorie...,True,True,successful,1583025192,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",4618.0,domestic
1,52,JinBucha is a new kind of Brewery in North Par...,"{""id"":307,""name"":""Drinks"",""slug"":""food/drinks""...",3461,US,the United States,1446051515,"{""id"":1468694331,""name"":""Jing Chen"",""slug"":""ji...",USD,$,...,jinbucha-a-modern-kombucha-tasting-room-in-nor...,https://www.kickstarter.com/discover/categorie...,False,True,failed,1450118057,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",3461.0,domestic
2,0,"""Taste The Scent of Tea"" Rose and Blanc Tea Ro...","{""id"":307,""name"":""Drinks"",""slug"":""food/drinks""...",0,US,the United States,1518159717,"{""id"":2026565095,""name"":""Giovanni"",""is_registe...",USD,$,...,tea-room-cafe-daily-and-herb-tea,https://www.kickstarter.com/discover/categorie...,False,False,failed,1520233260,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",0.0,domestic
3,2,Bringing Philly cheesesteaks (and other delici...,"{""id"":311,""name"":""Food Trucks"",""slug"":""food/fo...",25,US,the United States,1454217596,"{""id"":1629155736,""name"":""Glenn Edwards"",""is_re...",USD,$,...,help-two-brothers-open-a-food-truck,https://www.kickstarter.com/discover/categorie...,False,False,failed,1459885844,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",25.0,domestic
4,150,a frog plushie keychain and frog butt pin for ...,"{""id"":28,""name"":""Product Design"",""slug"":""desig...",3275,US,the United States,1571342290,"{""id"":932326000,""name"":""rainylune"",""slug"":""rai...",USD,$,...,son-the-frog-plushie-keychain-juicy-pin,https://www.kickstarter.com/discover/categorie...,True,False,successful,1574445600,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",3275.0,domestic


In [6]:
df.columns

Index(['backers_count', 'blurb', 'category', 'converted_pledged_amount',
       'country', 'country_displayable_name', 'created_at', 'creator',
       'currency', 'currency_symbol', 'currency_trailing_code',
       'current_currency', 'deadline', 'disable_communication', 'friends',
       'fx_rate', 'goal', 'id', 'is_backing', 'is_starrable', 'is_starred',
       'launched_at', 'location', 'name', 'permissions', 'photo', 'pledged',
       'profile', 'slug', 'source_url', 'spotlight', 'staff_pick', 'state',
       'state_changed_at', 'static_usd_rate', 'urls', 'usd_pledged',
       'usd_type'],
      dtype='object')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219688 entries, 0 to 219687
Data columns (total 38 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   backers_count             219688 non-null  int64  
 1   blurb                     219680 non-null  object 
 2   category                  219688 non-null  object 
 3   converted_pledged_amount  219688 non-null  int64  
 4   country                   219688 non-null  object 
 5   country_displayable_name  219688 non-null  object 
 6   created_at                219688 non-null  int64  
 7   creator                   219688 non-null  object 
 8   currency                  219688 non-null  object 
 9   currency_symbol           219688 non-null  object 
 10  currency_trailing_code    219688 non-null  bool   
 11  current_currency          219688 non-null  object 
 12  deadline                  219688 non-null  int64  
 13  disable_communication     219688 non-null  b

In [8]:
# check for duplicates of individual projects
dup_id = len(df[df.duplicated(subset='id')])
print(f"There are {dup_id} listed more than once.")

There are 27669 listed more than once.


In [9]:
# could get rid of the mostly null columns
mostly_null = ['friends', 'is_backing', 'is_starred', 'permissions']


In [10]:
df_dup_rows = df[df.duplicated(subset='id', keep=False)]
df_dup_rows.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55336 entries, 0 to 219672
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   backers_count             55336 non-null  int64  
 1   blurb                     55336 non-null  object 
 2   category                  55336 non-null  object 
 3   converted_pledged_amount  55336 non-null  int64  
 4   country                   55336 non-null  object 
 5   country_displayable_name  55336 non-null  object 
 6   created_at                55336 non-null  int64  
 7   creator                   55336 non-null  object 
 8   currency                  55336 non-null  object 
 9   currency_symbol           55336 non-null  object 
 10  currency_trailing_code    55336 non-null  bool   
 11  current_currency          55336 non-null  object 
 12  deadline                  55336 non-null  int64  
 13  disable_communication     55336 non-null  bool   
 14  frien

In [11]:
# discover how many rows contain duplicates
dup_rows = len(df_dup_rows)
print(f"There are {dup_rows} rows containing duplicates.")
print(f"That is an average of {(dup_rows/dup_id):.2f} copies per duplicated item.")

There are 55336 rows containing duplicates.
That is an average of 2.00 copies per duplicated item.


In [12]:
# check for number of unique 'category' items
# I suggest splitting 'category' into component columns - SW
df['category'].nunique()

171

In [13]:
# check state values
# combine canceled to failed?  & drop live?
# make booklean
print(df['state'].value_counts())
print("---"*15)

successful    128156
failed         76137
canceled        9138
live            6257
Name: state, dtype: int64
---------------------------------------------


In [14]:
# verify that 'is_starrable' largely associated with live campaigns
print(df['is_starrable'].value_counts())
print("---"*15)
print('')
print(df['state'].value_counts())
print("---"*15)

live_starrable = len(df.loc[(df['state'].isin(['live'])) & (df['is_starrable'])])

print(f'There are {live_starrable} rows are both classified as "live" and "is_starrable".')
print('Therefore a campaign must be live to be strarrable.')

False    213465
True       6223
Name: is_starrable, dtype: int64
---------------------------------------------

successful    128156
failed         76137
canceled        9138
live            6257
Name: state, dtype: int64
---------------------------------------------
There are 6223 rows are both classified as "live" and "is_starrable".
Therefore a campaign must be live to be strarrable.


In [16]:
# future data columns
'''
'spotlight' only applies to successfully funded projects,
'is_starrable' is only true with live campaigns,
'''

future_data = ['spotlight','is_starrable', 'staff_pick', 'backers_count', 'pledged', 'converted_pledged']


In [17]:
# extraneous columns

print(df['disable_communication'].value_counts())
print("---"*15)
print('')

# check for duplicate creators
print(f"There are {len(df[df.duplicated(subset='creator')])} listed more than once.")

'''
'disable communications' only contains the value 'false',
'fx_rate' is the foriegn exchange rate,
'country_displayable_name' duplicate information,
'creator', is high cardinality

'''
extra_columns = ['disable_communication','fx_rate', 'country_displayed_name', 'creator']

False    219688
Name: disable_communication, dtype: int64
---------------------------------------------

There are 669 listed more than once.


In [ ]:
# more extraneous columns to be removed

extras = ['photo', 'profile','urls', 'usd_pledged', 'state_changed_at', 'source_url']

In [57]:
df1=df.copy()

In [43]:
# split category into several columns
# must include map(eval)
df2 = df['category'].map(eval).apply(pd.Series)
df2.head()

,id,name,slug,position,parent_id,parent_name,color,urls
0,254,Performances,dance/performances,1,6.0,Dance,10917369,{'web': {'discover': 'http://www.kickstarter.c...
1,307,Drinks,food/drinks,4,10.0,Food,16725570,{'web': {'discover': 'http://www.kickstarter.c...
2,307,Drinks,food/drinks,4,10.0,Food,16725570,{'web': {'discover': 'http://www.kickstarter.c...
3,311,Food Trucks,food/food trucks,8,10.0,Food,16725570,{'web': {'discover': 'http://www.kickstarter.c...
4,28,Product Design,design/product design,5,7.0,Design,2577151,{'web': {'discover': 'http://www.kickstarter.c...


In [58]:
#Attempt to drop category column and add splits in one step - failed
# 
# pd.concat([df1.drop(['category'], axis=1), (df1['category'].map(eval).apply(pd.Series))], axis=1)
# df1.head()


,backers_count,blurb,category,converted_pledged_amount,country,country_displayable_name,created_at,creator,currency,currency_symbol,...,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
0,61,Support great art! Join us as we re-stage the ...,"{""id"":254,""name"":""Performances"",""slug"":""dance/...",4618,US,the United States,1579292017,"{""id"":2092817311,""name"":""Brooklyn Ballet"",""slu...",USD,$,...,revisionist-history-2,https://www.kickstarter.com/discover/categorie...,True,True,successful,1583025192,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",4618.0,domestic
1,52,JinBucha is a new kind of Brewery in North Par...,"{""id"":307,""name"":""Drinks"",""slug"":""food/drinks""...",3461,US,the United States,1446051515,"{""id"":1468694331,""name"":""Jing Chen"",""slug"":""ji...",USD,$,...,jinbucha-a-modern-kombucha-tasting-room-in-nor...,https://www.kickstarter.com/discover/categorie...,False,True,failed,1450118057,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",3461.0,domestic
2,0,"""Taste The Scent of Tea"" Rose and Blanc Tea Ro...","{""id"":307,""name"":""Drinks"",""slug"":""food/drinks""...",0,US,the United States,1518159717,"{""id"":2026565095,""name"":""Giovanni"",""is_registe...",USD,$,...,tea-room-cafe-daily-and-herb-tea,https://www.kickstarter.com/discover/categorie...,False,False,failed,1520233260,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",0.0,domestic
3,2,Bringing Philly cheesesteaks (and other delici...,"{""id"":311,""name"":""Food Trucks"",""slug"":""food/fo...",25,US,the United States,1454217596,"{""id"":1629155736,""name"":""Glenn Edwards"",""is_re...",USD,$,...,help-two-brothers-open-a-food-truck,https://www.kickstarter.com/discover/categorie...,False,False,failed,1459885844,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",25.0,domestic
4,150,a frog plushie keychain and frog butt pin for ...,"{""id"":28,""name"":""Product Design"",""slug"":""desig...",3275,US,the United States,1571342290,"{""id"":932326000,""name"":""rainylune"",""slug"":""rai...",USD,$,...,son-the-frog-plushie-keychain-juicy-pin,https://www.kickstarter.com/discover/categorie...,True,False,successful,1574445600,1.0,"{""web"":{""project"":""https://www.kickstarter.com...",3275.0,domestic


In [44]:
#drop extraneous category columns
df2=df2.drop(['id', 'slug', 'position','parent_id','color','urls'], axis=1)
df2.head()

,name,parent_name
0,Performances,Dance
1,Drinks,Food
2,Drinks,Food
3,Food Trucks,Food
4,Product Design,Design


In [60]:
# the main 15 categories are listed in 'parent_name'
# perhaps the earliest projects only had 'name'?
# 'slug' includes both pieces of info
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219688 entries, 0 to 219687
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   name         219688 non-null  object
 1   parent_name  211388 non-null  object
dtypes: object(2)
memory usage: 3.4+ MB
